In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import DBSCAN, KMeans

In [2]:
history = pd.read_csv('historical_data_supp_lots.csv')
history.head()

,Лот ID,Сумма во ВВ,Материал,Краткий текст,Курс,Краткий текст материала,Класс,Поставщик,Валюта
0,100000517882,137.00,1118280,Площадка самоклеящаяся Hyperline HW-3A,1.0,Площадка самоклеящаяся Hyperline HW-3A,2170,75422,RUB
1,100000517882,567.00,1118280,Площадка самоклеящаяся Hyperline HW-3A,1.0,Площадка самоклеящаяся Hyperline HW-3A,2170,94293,RUB
2,100000517882,567.00,1118280,Площадка самоклеящаяся Hyperline HW-3A,1.0,Площадка самоклеящаяся Hyperline HW-3A,2170,94293,RUB
3,100000517882,567.00,1118280,Площадка самоклеящаяся Hyperline HW-3A,1.0,Площадка самоклеящаяся Hyperline HW-3A,2170,94293,RUB
4,100000517882,619.76,1118280,Площадка самоклеящаяся Hyperline HW-3A,1.0,Площадка самоклеящаяся Hyperline HW-3A,2170,94501,RUB


In [3]:
suppliers = pd.read_csv('suppliers.csv')
suppliers.head()

,supplier_id,address_id
0,59,2016
1,1198,2313
2,1754,2417
3,5811,2038
4,8536,2038


In [4]:
addr = pd.read_csv('addresses.csv')
addr.head()

,address_id,address,latitude,longitude,geo_coords_confidence
0,0,"644099, Россия, г. Омск. ул. Фрунзе, д.54",54.340800,73.209800,1.0
1,1,"РФ, г. Омск, пр. Губкина, д.1",55.009071,73.297684,0.5
2,2,"г. Екатеринбург, ул. Сакко и Ванцетти, д.61",55.335305,61.311502,0.5
3,3,"Россия, 190900, Санкт-Петербург, box 1283",46.192326,5.996606,0.0
4,4,"г. Москва, Капотня, 2 квартал, д.1, корп. 3",43.362066,132.831962,0.5


In [5]:
suppliers = pd.merge(suppliers, addr, on='address_id', how='left')
history.rename(columns={'Поставщик':'supplier_id'}, inplace = True)
history = pd.merge(history, suppliers, how='left', on='supplier_id')
history.drop(columns=['Сумма во ВВ', 'Краткий текст', 'Курс', 'Краткий текст материала', 'Валюта', 'address_id', 'address', 'geo_coords_confidence'], inplace = True)
history.dropna(inplace=True)
history.sample(5)

,Лот ID,Материал,Класс,supplier_id,latitude,longitude
64077,100000595958,770000297604,21050102,47772,58.0000,56.2500
124007,100000595959,770000541019,21050101,58691,56.8575,60.6125
148100,100000627553,770000656809,21050101,47772,58.0000,56.2500
273978,100000529553,770001230051,210402,7120002608,58.2000,68.2667
99956,100000587671,770000467346,21050102,58691,56.8575,60.6125


In [6]:
target = history['Лот ID']
cat_features = ['Материал', 'Класс', 'supplier_id', 'Лот ID']
numeric_features = ['latitude', 'longitude']

In [7]:
class Printer():
  def __init__(self):
    pass
  def fit(self, X, y):
    print('Hi')
    return self
  def transform(self, X):
    return X

In [8]:
ct = ColumnTransformer([('encode_cats', OneHotEncoder(), cat_features),('scale', StandardScaler(), numeric_features)], remainder='passthrough')
model = make_pipeline(
    Printer(),
    ct,
    Printer(),
    KMeans(400)
)

In [9]:
preds = model.fit_predict(history)

Hi
Hi


In [22]:
historynew = history.assign(kmeans_clust=preds)
historynew.to_csv('KMeansclusters.csv', index=False)